# Imports

In [34]:
from nba_api.stats.endpoints import leaguegamelog
import model.dataset.data as data
import model.dataset.config as config

import model.dataset.season_game as sg
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [ ]:
from nba_api.stats.endpoints import leaguedashteamstats

In [ ]:
leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]

In [ ]:
rankings = data.load_rankings()

In [37]:
list(raw_season_games.columns)

['SEASON_ID',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'PTS_AGAINST',
 'W_L',
 'W_L_CUM',
 'PTS_CUM',
 'PTS_AGAINST_CUM',
 'W_L_L10_CUM',
 'PTS_L10_CUM',
 'PTS_AGAINST_L10_CUM',
 'SEASON',
 'LOCATION',
 'UNIQUE_MATCHUP',
 'FGM_ML10',
 'FGA_ML10',
 'FG_PCT_ML10',
 'FG3M_ML10',
 'FG3A_ML10',
 'FG3_PCT_ML10',
 'FTM_ML10',
 'FTA_ML10',
 'FT_PCT_ML10',
 'OREB_ML10',
 'DREB_ML10',
 'REB_ML10',
 'AST_ML10',
 'STL_ML10',
 'BLK_ML10',
 'TOV_ML10',
 'PF_ML10',
 'PTS_ML10',
 'PLUS_MINUS_ML10',
 'W_L_CUM_ML10',
 'PTS_CUM_ML10',
 'PTS_AGAINST_CUM_ML10']

# Load local dataset

## Raw Season Games

In [35]:
raw_season_games = pd.read_feather(config.RAW_SEASON_GAMES_DS)
raw_season_games.set_index(["GAME_ID"], inplace=True)
raw_season_games.sort_values(by=['GAME_DATE', 'GAME_ID'], inplace=True)

In [6]:
len(list(raw_season_games.SEASON_ID.unique()))

18

In [ ]:

def filter(df):
    return (df.TEAM_ID == 1610612755) & (df.SEASON_ID.str[-4:] == '2017')
    
raw_season_games[filter(raw_season_games)].head(18)

## Season games

In [36]:
season_games = pd.read_feather(config.SEASON_GAMES_DS)
season_games.set_index(["GAME_ID"], inplace=True)
season_games.sort_values(by=['GAME_DATE', 'GAME_ID'], inplace=True)

In [ ]:
def filter(df):
    return ((df.TEAM_ID_HOME == 1610612755) | (df.TEAM_ID_AWAY == 1610612755)) & (df.SEASON_ID.str[-4:] == '2017')
    
#season_games[['PTS_HOME', 'MATCHUP_HOME', 'MATCHUP_AWAY', 'PTS_AWAY', 'HOME_POINT_SPREAD']][filter(season_games)].head(18)
season_games[filter(season_games)].head(18)


In [ ]:
sg.create_calculate_fields(season_games)
#season_games['UNIQUE_MATCHUP_AWAY']

In [ ]:
list(season_games.columns)

# Matchups games

In [ ]:
def matchup_field(row):
    if " vs. " in row.MATCHUP:
        return '-'.join(
            sorted(
                row.MATCHUP.split(" vs. ")
            ))
    else:
        return '-'.join(
            sorted(
                row.MATCHUP.split(" @ ")
            ))                                 

def matchup_field_by_id(row):
    #isplay(row.name)
    game_df = raw_season_games.loc[row.name]
    #ther_team_id = game_df[game_df.TEAM_ID != row.TEAM_ID]['TEAM_ID'].values[0]
    return '-'.join(map(str,sorted(game_df.TEAM_ID.values.tolist())))
#'-'.join(sorted(['BKN', 'ATL']))    
raw_season_games["UNIQUE_MATCHUP"] = raw_season_games.apply(lambda row: matchup_field_by_id(row), axis=1)
#                                                   '-'.join(
#                                                       sorted(
#                                                           [row.TEAM_ABBREVIATION_HOME
#                                                            , row.TEAM_ABBREVIATION_AWAY])),
#                                                       ,axis=1)

"PHI @ ORL".split(" @ ") + "PHI vs. ORL".split(" vs. ")
"PHI @ ORL".split(" @ ") + "PHI @ ORL".split(" vs. ")
" vs. " in "PHI @ ORL"
len(raw_season_games["UNIQUE_MATCHUP"].unique())

# Test transformations

In [100]:
cols = ['SEASON_ID',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'FGM','FGM_MEAN',
 'FGA','FGA_MEAN',
 'FG_PCT','FG_PCT_MEAN',
 'FG3M','FG3M_MEAN',
 'FG3A','FG3A_MEAN',
 'FG3_PCT','FG3_PCT_MEAN',
 'FTM','FTM_MEAN',
 'FTA','FTA_MEAN',
 'FT_PCT','FT_PCT_MEAN',
 'OREB','OREB_MEAN',
 'DREB','DREB_MEAN',
 'REB','REB_MEAN',
 'AST','AST_MEAN',
 'STL','STL_MEAN',
 'BLK','BLK_MEAN',
 'TOV','TOV_MEAN',
 'PF','PF_MEAN',
 'PTS','PTS_MEAN',
 'PLUS_MINUS','PLUS_MINUS_MEAN',
 'W_L',
 'W_L_CUM',
 'W_L_L10_CUM',
 'SEASON',
 'LOCATION']

def expanding_mean(x):
    return x.rolling(window=82, min_periods=0).mean()

In [ ]:
df_17 = raw_season_games[raw_season_games.SEASON_ID.str[-4:] == '2017']
df_18  = raw_season_games[raw_season_games.SEASON_ID.str[-4:] == '2018']
pd.concat([df_17, df_18])
#leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
#                                           , season=2017).get_data_frames()[0] \
#                                            .set_index('GAME_ID').sort_values(by=['GAME_DATE'])
#.set_index('GAME_ID', inplace=True)
#raw_season_games

In [10]:
next_season = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
                                        , season="2017-18").get_data_frames()[0]

next_season.isna().sum().sum()

0

In [32]:
def pts_against(row, df):
    against_row = df[((df.index == row.name) & (df.TEAM_ID != row.TEAM_ID))].iloc[0]
    return against_row.PTS

cond = (raw_season_games.SEASON_ID.str[-4:] == '2018')
# cond = ((raw_season_games.TEAM_ABBREVIATION == 'MIL') 
# #         | (raw_season_games.TEAM_ABBREVIATION == 'ATL')
#         ) & (raw_season_games.SEASON_ID.str[-4:] == '2018')

next_season = raw_season_games[cond]
#next_season
next_season["PTS_AGAINST"] = next_season.apply(lambda row: pts_against(row, next_season), axis=1)
print('Done')

Done


/home/nathaniel/anaconda3/envs/nba-predictor/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [33]:
next_season

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,W_L,W_L_CUM,PTS_CUM,W_L_L10_CUM,PTS_L10_CUM,SEASON,LOCATION,UNIQUE_MATCHUP,FGM_ML10,FGA_ML10,FG_PCT_ML10,FG3M_ML10,FG3A_ML10,FG3_PCT_ML10,FTM_ML10,FTA_ML10,FT_PCT_ML10,OREB_ML10,DREB_ML10,REB_ML10,AST_ML10,STL_ML10,BLK_ML10,TOV_ML10,PF_ML10,PTS_ML10,PLUS_MINUS_ML10,W_L_CUM_ML10,PTS_CUM_ML10,PTS_AGAINST
GAME_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0021800001,22018,1610612755,PHI,Philadelphia 76ers,2018-10-16,PHI @ BOS,L,240,34,87,0.391,5,26,0.192,14,23,0.609,6,41,47,18,8,5,16,20,87,-18,-1,NaN,NaN,NaN,NaN,2018,AWAY,1610612738-1610612755,37.3,84.8,0.4411,10.3,28.1,0.3646,15.4,20.1,0.7629,9.7,32.9,42.6,22.9,9.6,5.3,18.5,22.2,100.3,-6.5,-6.0,1003.0,105
0021800001,22018,1610612738,BOS,Boston Celtics,2018-10-16,BOS vs. PHI,W,240,42,97,0.433,11,37,0.297,10,14,0.714,12,43,55,21,7,5,15,20,105,18,1,NaN,NaN,NaN,NaN,2018,HOME,1610612738-1610612755,38.2,84.4,0.4525,10.9,29.5,0.3665,19.5,24.7,0.7933,9.6,33.9,43.5,23.3,8.3,4.7,17.4,19.9,106.8,6.5,6.0,1068.0,87
0021800002,22018,1610612760,OKC,Oklahoma City Thunder,2018-10-16,OKC @ GSW,L,240,33,91,0.363,10,37,0.270,24,37,0.649,16,29,45,21,12,6,15,21,100,-8,-1,NaN,NaN,NaN,NaN,2018,AWAY,1610612744-1610612760,38.5,89.3,0.4310,7.9,27.4,0.2851,20.0,28.4,0.7034,13.6,32.3,45.9,19.1,9.0,4.4,16.4,20.9,104.9,-9.6,-6.0,1049.0,108
0021800002,22018,1610612744,GSW,Golden State Warriors,2018-10-16,GSW vs. OKC,W,240,42,95,0.442,7,26,0.269,17,18,0.944,17,41,58,28,7,7,21,29,108,8,1,NaN,NaN,NaN,NaN,2018,HOME,1610612744-1610612760,42.3,85.4,0.4949,12.0,30.7,0.3911,17.9,23.5,0.7718,8.8,34.0,42.8,28.8,9.9,7.0,15.9,21.0,114.5,9.6,6.0,1145.0,100
0021800003,22018,1610612749,MIL,Milwaukee Bucks,2018-10-17,MIL @ CHA,W,240,42,85,0.494,14,34,0.412,15,20,0.750,11,46,57,26,5,4,21,25,113,1,1,NaN,NaN,NaN,NaN,2018,AWAY,1610612749-1610612766,40.0,83.1,0.4841,9.3,23.6,0.3758,14.0,17.5,0.7856,7.5,34.9,42.4,25.8,4.7,5.0,12.8,21.0,103.3,-0.1,0.0,1033.0,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0021801228,22018,1610612743,DEN,Denver Nuggets,2019-04-10,DEN vs. MIN,W,240,39,87,0.448,10,33,0.303,11,17,0.647,12,41,53,23,6,4,13,12,99,4,1,25.0,8976.0,-2.0,1023.0,2018,HOME,1610612743-1610612750,41.1,86.2,0.4790,12.5,31.9,0.3928,14.3,18.3,0.7783,10.7,32.9,43.6,27.1,5.3,5.1,14.3,19.0,109.0,0.2,0.0,1090.0,95
0021801229,22018,1610612746,LAC,LA Clippers,2019-04-10,LAC vs. UTA,W,265,54,106,0.509,12,28,0.429,23,30,0.767,12,40,52,34,7,5,12,27,143,6,1,13.0,9299.0,2.0,1158.0,2018,HOME,1610612746-1610612762,37.7,83.2,0.4554,8.3,26.8,0.3060,18.6,24.3,0.7464,9.5,31.9,41.4,18.3,9.1,4.0,12.7,20.3,102.3,-2.5,-2.0,1023.0,137
0021801229,22018,1610612762,UTA,Utah Jazz,2019-04-10,UTA @ LAC,L,265,47,106,0.443,14,35,0.400,29,33,0.879,17,40,57,31,8,11,17,24,137,-6,-1,19.0,9024.0,6.0,1171.0,2018,AWAY,1610612746-1610612762,38.9,83.1,0.4685,10.0,27.4,0.3673,17.0,22.5,0.7570,10.0,34.9,44.9,23.1,7.3,5.1,14.4,20.9,104.8,2.5,2.0,1048.0,143


In [28]:
# next_season = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
#                                        , season="2017-18").get_data_frames()[0].set_index('GAME_ID').sort_values(by=['GAME_DATE'])

#cond = (raw_season_games.SEASON_ID.str[-4:] == '2018')
# cond = ((raw_season_games.TEAM_ABBREVIATION == 'MIL') 
# #         | (raw_season_games.TEAM_ABBREVIATION == 'ATL')
#         ) & (raw_season_games.SEASON_ID.str[-4:] == '2018')

# next_season = raw_season_games[cond]
# next_season

#raw_season_games.loc['0021800003']
#raw_season_games.TEAM_ID == '1610612766'

raw_season_games[((raw_season_games.index == '0021800003') & (raw_season_games.TEAM_ID == 1610612766))].iloc[0].PTS
#

 #
#next_seasabson.reset_index()
#.sort_values(by=['GAME_DATE', 'GAME_ID'])|

#grouped = next_season.groupby(by=["TEAM_ID", "GAME_ID"]).count()

#0021800023 4200
#print(next_season.loc['0021800003']["FGM"])
#res = next_season.groupby(by=["TEAM_ID"])[['FGM']].expanding().mean().groupby(level=0).shift(1)
#.reset_index(level=0)
#res.loc['0021800023']["FGM"]
#res

# next_season[['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB',
#              'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']] \
# .groupby(next_season["TEAM_ID"], group_keys=False).shift(1) \
#             .rolling(window=82, min_periods=0).mean()
#.apply(expanding_mean)
#season_games_mean
#next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_MEAN'],
#                                on=['GAME_ID'])
# next_season = pd.concat([next_season, season_games_mean], axis=1, sort=False)
# next_season[next_season.TEAM_ABBREVIATION == 'MIL'][cols].head(300)


112

In [ ]:
# .reset_index(level=0)
# season_games_mean
# season_games_mean.isnull().sum().sum()
#next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_MEAN'], on=['GAME_ID', 'TEAM_ID'])

# season_l10_games_mean = next_season.groupby(by=["TEAM_ID"]).shift(1, fill_value=0)[
#             ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
#              'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']] \
#             .rolling(window=10, min_periods=1).mean()
# season_l10_games_mean = season_l10_games_mean.assign(key=next_season['TEAM_ID'])
# season_l10_games_mean

#result = test_season_games.groupby(by=["TEAM_ID", "UNIQUE_MATCHUP"])[['W_L']]\
#.expanding().sum().reset_index(level=0)

#print(next_season[(next_season.index == '0021700017') & (next_season.TEAM_ID == 1610612737)].iloc[0])
#cond = (result.TEAM_ID == 1610612737) & (result.UNIQUE_MATCHUP == '1610612737-1610612738')
#result[cond]

#.reset_index(level=0)
#.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})
#next_season
# result
# compare_df = pd.DataFrame()
# print(result["TEAM_ID"].shape)
# print(next_season["TEAM_ID"].shape)
# compare_df['team_id'] = next_season["TEAM_ID"]
# compare_df['result_team_id'] = result["TEAM_ID"]

# comparison = result["TEAM_ID"].values == next_season["TEAM_ID"].values
# equal_arrays = comparison.all()

# np.setdiff1d(result["TEAM_ID"].values, next_season["TEAM_ID"].values)
# diff = result["TEAM_ID"].values - next_season["TEAM_ID"].values
# diff
# #compare_df['different'] = np.where( == , True, False)
# #compare_df
raw_season_games[raw_season_games.UNIQUE_MATCHUP == '1610612737-1610612738']

# Fetch and transform season games

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguegamelog
import model.dataset.data as data

seasons = data.load_seasons().SEASON.unique()
# season_games = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
#                                             , season=seasons[-11]).get_data_frames()[0].sort_values(by=['SEASON_ID', 'GAME_DATE', 'GAME_ID'])
season_games = pd.DataFrame()
raw_season_games = pd.DataFrame()
for season in seasons[-11:]:
    next_season = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
                                           , season=season).get_data_frames()[0] \
                                            .set_index('GAME_ID').sort_values(by=['GAME_DATE'])
                                            #.sort_values(by=['SEASON_ID', 'GAME_DATE', 'GAME_ID'])
    raw_season_games = pd.concat([raw_season_games, next_season])
    
    next_season.dropna(inplace=True)
    next_season["W_L"] = np.where(next_season['WL'] == 'W', 1, -1)
    season_games_sum = next_season.groupby(by=["TEAM_ID"])[['W_L']]\
    .expanding().sum().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_sum, suffixes=['', '_CUM'], on=['GAME_ID', 'TEAM_ID'])
    
    season_games_mean = next_season.groupby(by=["TEAM_ID"])[['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]\
    .expanding().mean().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_MEAN'], on=['GAME_ID', 'TEAM_ID'])
    
    next_season.groupby(by=["TEAM_ID"])[['W_L', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]\
    .rolling(window=10, min_periods=1).median().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_L10'], on=['GAME_ID', 'TEAM_ID'])
    
    season_home_rows = next_season[next_season.MATCHUP.str.contains('vs.')]
    season_away_rows = next_season[next_season.MATCHUP.str.contains('@')]
    # Join every row to all others with the same game ID.
    joined = pd.merge(season_home_rows, season_away_rows, suffixes=['_HOME', '_AWAY'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_HOME != joined.TEAM_ID_AWAY]
    season_games = pd.concat([season_games, result])

In [ ]:
pd.options.display.max_rows = 999


season_games[season_games.isnull().any(axis=1)]

In [ ]:
#season_games['SEASON_YEAR'] = season_games.SEASON_ID.str[1:]
season_games.drop(columns=['VIDEO_AVAILABLE_HOME', 'VIDEO_AVAILABLE_AWAY'], axis=1, inplace=True)

# Test fetched season games

In [ ]:
def create_filter(team, season_id, game_date):
    return (season_games.GAME_DATE < game_date) & \
            (season_games.SEASON_ID == season_id) & \
            ((season_games.TEAM_ID_HOME == team) | (season_games.TEAM_ID_AWAY == team))
    

In [ ]:
row.index

In [ ]:


row = season_games.iloc[200]
print(row)
home = row.TEAM_ID_HOME
away = row.TEAM_ID_AWAY
season_id = row.SEASON_ID
game_date = row.GAME_DATE

home_games = season_games[create_filter(home, season_id, game_date)]
away_games = season_games[create_filter(home, season_id, game_date)]

home_games.columns

#season_games.expanding().mean()


# Testing teamgamelogs endpoint

In [ ]:
from nba_api.stats.endpoints import teamgamelogs

# Query for games where the Celtics were playing
gamefinder = teamgamelogs.TeamGameLogs(team_id_nullable=celtics_id)
all_games = teamgamelogs.TeamGameLogs(season_type_nullable="Regular Season"
                                          , season_nullable='2010-11').get_data_frames()[0]
teamgamelogs.SeasonNullable.current_season
# The first DataFrame of those returned is what we want.
all_games.head()


In [ ]:
all_games.groupby(all_games.SEASON_YEAR)[['GAME_ID']].count()
#.loc['2015':]